In [150]:
import os
import re
import math
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from copy import copy

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

from xgboost import XGBClassifier
import xgboost as xgb
from tabulate import tabulate
# from matplotlib import pyplot as plt

DIRPATH = os.path.abspath(".")

In [10]:
transaction = pd.read_csv(DIRPATH + "/data/账户交易信息.csv")
account = pd.read_csv(DIRPATH + "/data/账户静态信息.csv")
label = pd.read_csv(DIRPATH + "/data/训练集标签.csv")
test_df = pd.read_csv(DIRPATH + "/data/test_dataset.csv")

In [21]:
table_shape = []
for dfi in ["transaction", "account", "label", "test_df"]:
    table_shape.append([dfi, eval(dfi).shape])
print(tabulate(table_shape, headers=["Name", "Dataframe Shape"], tablefmt="github"))

| Name        | Dataframe Shape   |
|-------------|-------------------|
| transaction | (816270, 12)      |
| account     | (6000, 5)         |
| label       | (1200, 2)         |
| test_df     | (4800, 1)         |


In [129]:
transaction_info = pd.merge(transaction, account, on="zhdh")
transaction_info["jysjf"] = pd.to_datetime(transaction_info["jyrq"] + " " + transaction_info["jysj"])
transaction_info["jysjf"] = transaction_info["jysjf"].astype(int)/ 10**9

transaction_info["khrq"] = pd.to_datetime(transaction_info["khrq"])
transaction_info["khrqf"] = transaction_info["khrq"].astype(int)/10**9 
transaction_info.head()

,jylsxh,zhdh,dfzh,jdbj,jyje,zhye,dfhh,jyrq,jysj,jyqd,zydh,dfmccd,khrq,khjgdh,xb,年龄,jysjf,khrqf
0,5D252156AE9F6B6595A1C56F56D4F91C,86C379D938234BAA,14BEFED1370B730A,0,310.00,57806.83,834E1F06,2020-03-01,00:18:06,E96ED478,4E0CB6FB,45,2014-02-28,37BC2F75,0,28,1.583022e+09,1.393546e+09
1,D6AC72331F6A360F6879162882A62A56,86C379D938234BAA,BFA099089E1C459E,0,1000.00,56806.83,469F7281,2020-03-01,13:38:14,E96ED478,4E0CB6FB,36,2014-02-28,37BC2F75,0,28,1.583070e+09,1.393546e+09
2,B43E51D87E41467A6A1B4C2264304882,86C379D938234BAA,BC2042D38BEE2B23,0,20.00,56786.83,AACC16EA,2020-03-01,14:23:53,757B505C,0EFE3284,12,2014-02-28,37BC2F75,0,28,1.583073e+09,1.393546e+09
3,FE74B464378C80D448D200DF694C7629,86C379D938234BAA,0F76A363EACBDEAF,1,32.56,56819.39,D41D8CD9,2020-03-01,14:56:24,757B505C,35240722,0,2014-02-28,37BC2F75,0,28,1.583075e+09,1.393546e+09
4,4E39792E24BF493EADDFE3892399B24D,86C379D938234BAA,1F77C9FDBDBE78D0,0,80.00,56739.39,469F7281,2020-03-01,18:16:24,E96ED478,4E0CB6FB,18,2014-02-28,37BC2F75,0,28,1.583087e+09,1.393546e+09


In [130]:
df_all = pd.merge(label, transaction_info, on="zhdh")
df_all.head()

,zhdh,black_flag,jylsxh,dfzh,jdbj,jyje,zhye,dfhh,jyrq,jysj,jyqd,zydh,dfmccd,khrq,khjgdh,xb,年龄,jysjf,khrqf
0,2029FF26D4E2CA79,0,402AE7983AC34E7C39AA2A97A0FEA554,B3CF1FD3828A2A14,1,3052.84,365859.88,AB4158ED,2020-03-01,07:43:00,892C91E0,56D9BA4E,0,2018-03-28,698D51A1,1,40,1.583049e+09,1.522195e+09
1,2029FF26D4E2CA79,0,AA4DC9355822E97361B6B8E2C9AEEBE5,75A9DC6D78C89B6C,1,60000.00,425859.88,98CB458A,2020-03-02,00:34:51,621461AF,A3C65C29,6,2018-03-28,698D51A1,1,40,1.583109e+09,1.522195e+09
2,2029FF26D4E2CA79,0,F8710ED2DFE4DD313EA18041C3D5D2C8,B2B22C17E3128454,0,7008.39,418851.49,AB4158ED,2020-03-11,04:08:30,892C91E0,D2716515,0,2018-03-28,698D51A1,1,40,1.583900e+09,1.522195e+09
3,2029FF26D4E2CA79,0,258FE9A18B9A3A9AF470E574CB52042F,75A9DC6D78C89B6C,1,20000.00,438851.49,98CB458A,2020-03-14,15:50:01,621461AF,A3C65C29,6,2018-03-28,698D51A1,1,40,1.584201e+09,1.522195e+09
4,2029FF26D4E2CA79,0,A68F8638FDB591B88C9346AAAC5A2A9B,75A9DC6D78C89B6C,1,20000.00,458851.49,98CB458A,2020-03-20,18:32:01,621461AF,A3C65C29,6,2018-03-28,698D51A1,1,40,1.584729e+09,1.522195e+09


In [131]:
account_num = len(set(transaction_info["zhdh"])) 
nolabel_account_num = account_num - len(set(transaction_info["zhdh"]) & set(label["zhdh"]))
print("共有账户：{}个\n有交易信息，但是没有label的账户有： {}个".format(account_num, nolabel_account_num))

print(len(set(transaction_info["zhdh"]) - set(label["zhdh"]) - set(test_df["zhdh"])))
print("无label的 {}个 账户 等同 测试数据集中的账户".format(nolabel_account_num))

共有账户：6000个
有交易信息，但是没有label的账户有： 4800个
0
无label的 4800个 账户 等同 测试数据集中的账户


In [142]:
# 选择变量
p_tran_id = ["zhdh", "jylsxh", "zydh"]
p_time = ["khrq", "jyrq", "jysj"]
p_cat = ["dfzh", "dfhh", "jyqd", "khjgdh"]

df_train = df_all.copy()
df_train = df_train.drop(columns=p_tran_id + p_time)


df_test_X = pd.merge(test_df, transaction_info, on="zhdh")
df_test_X = df_test_X.drop(columns=p_tran_id + p_time)

# for var in p_cat:
#     df_train[var] = df_train.astype("category")
#     df_test_X[var] = df_test_X.astype("category")

df_test_X.head()

,dfzh,jdbj,jyje,zhye,dfhh,jyqd,dfmccd,khjgdh,xb,年龄,jysjf,khrqf
0,7495D485C0213E9C,0,19.00,2879.34,7495D485C0213E9C,7495D485C0213E9C,16,7495D485C0213E9C,1,41,1.583647e+09,1.544573e+09
1,14BEFED1370B730A,0,15.00,2864.34,14BEFED1370B730A,14BEFED1370B730A,45,14BEFED1370B730A,1,41,1.583828e+09,1.544573e+09
2,14BEFED1370B730A,0,141.28,2723.06,14BEFED1370B730A,14BEFED1370B730A,45,14BEFED1370B730A,1,41,1.583828e+09,1.544573e+09
3,14BEFED1370B730A,0,492.00,2231.06,14BEFED1370B730A,14BEFED1370B730A,16,14BEFED1370B730A,1,41,1.584102e+09,1.544573e+09
4,14BEFED1370B730A,0,50.00,2181.06,14BEFED1370B730A,14BEFED1370B730A,16,14BEFED1370B730A,1,41,1.584104e+09,1.544573e+09


In [157]:
df_train.describe()

,black_flag,jdbj,jyje,zhye,dfmccd,xb,年龄,jysjf,khrqf
count,165311.000000,165311.000000,1.653110e+05,1.653110e+05,165311.000000,165311.000000,165311.000000,1.653110e+05,1.653110e+05
mean,0.256141,0.358893,5.281663e+03,3.797571e+04,19.711725,0.394342,36.120397,1.587019e+09,1.462218e+09
std,0.436502,0.479677,5.577935e+04,2.601738e+05,13.589711,0.488710,11.059980,2.296024e+06,1.095125e+08
min,0.000000,0.000000,-5.000000e+05,0.000000e+00,0.000000,0.000000,12.000000,1.583022e+09,8.989920e+08
25%,0.000000,0.000000,1.600000e+01,6.311150e+02,9.000000,0.000000,28.000000,1.584965e+09,1.394928e+09
50%,0.000000,0.000000,1.585000e+02,4.314420e+03,16.000000,0.000000,35.000000,1.587073e+09,1.483229e+09
75%,1.000000,1.000000,1.547305e+03,2.208128e+04,28.000000,1.000000,41.000000,1.588961e+09,1.553386e+09
max,1.000000,1.000000,8.600000e+06,1.029000e+07,97.000000,1.000000,83.000000,1.590970e+09,1.590278e+09


In [158]:
X_train = df_train.drop(columns=["black_flag"])
y_train = df_train["black_flag"]

# cat_attribs = p_cat
# full_pipeline = ColumnTransformer([('cat', OneHotEncoder(handle_unknown='ignore'), cat_attribs)], remainder='passthrough')

# encoder = full_pipeline.fit(X_train)
# X_train = encoder.transform(X_train)

xgb = XGBClassifier(tree_method="gpu_hist", enable_categorical=True)

xgb.fit(X_train, y_train)

# data = []
# for i in range(len(X.columns)):
#     data.append([X.columns[i], xgb.feature_importances_[i]])
# sorted_data = sorted(data, key=lambda x: x[1], reverse=True)


# print(tabulate(sorted_data[:10], headers=['Features(前十个)', "Importance"]))

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:dfzh: object, dfhh: object, jyqd: object, khjgdh: object